## Bài 2

<b> Mô tả bài toán </b>

Khi làm quản trò, anh Huy thường sử dụng 2 viên xúc sắc khác nhau, viên đầu tiên là một viên xúc xắc cân bằng, mọi mặt đều có cùng xác suất. Viên thứ hai là một viên xúc xắc lỗi, khi tung sẽ có 50% xác suất ra mặt số 6 và 10% xác suất ra mỗi mặt còn lại. Mỗi lần tung, anh sẽ chọn 1 trong 2 viên xúc xắc để tung. Người chơi không thể biết anh đã tung viên nào, chỉ biết được lần tung đó ra mặt nào.

Ngoài ra, nếu ở lần tung này, anh Huy sử dụng viên xúc xắc cân bằng, thì có $80%$ khả năng anh sẽ tiếp tục sử dụng viên xúc xắc này cho lần tung tiếp theo ($20%$ còn lại anh sẽ đổi sang dùng viên lỗi). Con số này là $30%$ đối với viên lỗi ($70%$ đổi sang dùng viên cân bằng).

a. Mô hình hoá tình huống trên bằng một mô hình Markov ẩn. Cho biết các tham số của mô hình này.

b. Sinh ngẫu nhiên một chuỗi $T = 100$ lần tung đúng theo mô tả trên.

c. Sử dụng thuật toán Viterbi để dự đoán viên xúc xắc được dùng cho mỗi lần tung. Độ chính xác của dự đoán này là bao nhiêu? Hãy lặp lại thí nghiệm này nhiều lần nếu cần thiết. Báo cáo và nhận xét kết quả thu được.

d. Giả sử bạn là một người chơi, hãy sử dụng thuật toán Baum-Welch để ước lượng các tham số cho mô hình Markov ẩn. Hãy lặp lại thí nghiệm này nhiều lần nếu cần thiết. Báo cáo kết quả thu được.

-----------------------------------------------------

<b> a. Mô hình hoá bài toán </b>

Tại một thời điểm $t$ bất kì, người chơi không thể biết được anh Huy chọn viên xúc xắc nào để gieo mà chỉ có thể biết được kết quả của phép gieo xúc xắc. Vì thế, ta có thể quy định:
- Viên xúc xắc được chọn tại một thời điểm là trạng thái ẩn (hidden state) có giá trị là $I$ hoặc $II$.
- Số điểm thu được từ phép gieo xúc xắc là trạng thái có thể quan sát (visible state) có giá trị trong khoảng $[1, 6]$.

Hai viên xúc xắc của anh Huy:
- Viên xúc xắc đầu tiên ($I$) là một viên xúc xắc ra các mặt từ 1 đến 6 là như nhau.
- Viên xúc xắc thứ hai  ($II$) là một viên xúc xắc không cân bằng, 50% ra mặt số 6 và 10% cho ra mỗi mặt còn lại.

Từ đây ta có thể suy ra ma trận quan hệ giữa các trạng thái ẩn và trạng thái quan sát (emission matrix):

|    | $1$   | $2$   | $3$   | $4$  | $5$   | $6$   |
|----|-----|-----|-----|-----|-----|-----|
| $I$  | $\frac{1}{6}$ | $\frac{1}{6}$ | $\frac{1}{6}$ | $\frac{1}{6}$ | $\frac{1}{6}$ | $\frac{1}{6}$ |
| $II$ | $0.1$ | $0.1$ | $0.1$ | $0.1$ | $0.1$ | 0.5 |

Ngoài ra, xét tại thời điểm $t$ bất kì:
- Anh Huy chọn viên xúc xắc $I$, lượt tiếp theo:
- - Xác xuất anh Huy tiếp tục sử dụng viên này là: 80%
- - Xác suất anh Huy sử dụng viên thứ $II$ cho lượt tiếp theo là: 20%
- Anh Huy chọn viên xúc xắc $II$:
- - Xác xuất anh Huy tiếp tục sử dụng viên này là: 30%
- - Xác suất anh Huy sử dụng viên thứ $I$ cho lượt tiếp theo là: 70%

Từ đây ta có thể suy ra ma trận chuyển tiếp giữa các trạng thái ẩn (transition matrix):

|    | $I$   | $II$  |
|----|-----|-----|
| $I$  | $0.8$ | $0.2$ |
| $II$ | $0.7$ | $0.3$ |


Và anh Huy không cho biết gì thêm, giá sử xác suất ban đầu cho việc chọn mỗi viên xúc xắc là như nhau:

|    | $I$   | $II$  |
|----|-----|-----|
| $p$  | $0.5$ | $0.5$ |


In [2]:
import numpy as np
import hmm

In [3]:
emission_matrix = [
    [1 / 6  , 1 / 6 , 1 / 6 , 1 / 6 , 1 / 6 , 1 / 6 ],
    [0.1    , 0.1   , 0.1   , 0.1   , 0.1   , 0.5   ]
]

transition_matrix = [
    [0.8, 0.2],
    [0.7, 0.3]
]

pi_matrix = [
    0.5, 0.5
]

dice_set = [0, 1]

-----------------------------------------------------

<b> b. Mô phỏng 100 lần tung. </b>

Sử dụng hàm random với các trọng số được mô tả ở câu trên.

(Để dễ dàng hơn trong việc lập trình, chỉ số của hai viên xúc sắc sẽ là $0$ và $1$ - thay vì $I$ và $II$ trong mô tả)

In [80]:
T = 100

current_dice = None

dice_chose = [None for _ in range(T)]
result = [None for _ in range(T)]

for t in range(T):
    if t == 0:
        dice_no = np.random.choice(dice_set, p = pi_matrix)
        dice_chose[t] = dice_no
        result[t] = np.random.choice(np.arange(0, 6), p = emission_matrix[dice_no])
    else:
        dice_no = np.random.choice(dice_set, p = transition_matrix[dice_no])
        dice_chose[t] = dice_no
        result[t] = np.random.choice(np.arange(0, 6), p = emission_matrix[dice_no])

for t in range(T):
    print(f'{t + 1}\t Chose: {dice_chose[t] + 1} - Score: {result[t] + 1}')

1	 Chose: 1 - Score: 6
2	 Chose: 2 - Score: 6
3	 Chose: 1 - Score: 3
4	 Chose: 1 - Score: 6
5	 Chose: 1 - Score: 6
6	 Chose: 1 - Score: 4
7	 Chose: 1 - Score: 4
8	 Chose: 1 - Score: 6
9	 Chose: 1 - Score: 1
10	 Chose: 1 - Score: 5
11	 Chose: 1 - Score: 2
12	 Chose: 1 - Score: 4
13	 Chose: 2 - Score: 1
14	 Chose: 2 - Score: 6
15	 Chose: 2 - Score: 2
16	 Chose: 2 - Score: 6
17	 Chose: 2 - Score: 6
18	 Chose: 2 - Score: 2
19	 Chose: 1 - Score: 2
20	 Chose: 2 - Score: 6
21	 Chose: 1 - Score: 6
22	 Chose: 1 - Score: 2
23	 Chose: 1 - Score: 1
24	 Chose: 1 - Score: 5
25	 Chose: 1 - Score: 4
26	 Chose: 1 - Score: 3
27	 Chose: 2 - Score: 6
28	 Chose: 1 - Score: 3
29	 Chose: 2 - Score: 6
30	 Chose: 2 - Score: 5
31	 Chose: 2 - Score: 6
32	 Chose: 1 - Score: 5
33	 Chose: 1 - Score: 6
34	 Chose: 1 - Score: 2
35	 Chose: 2 - Score: 6
36	 Chose: 1 - Score: 6
37	 Chose: 1 - Score: 1
38	 Chose: 1 - Score: 6
39	 Chose: 1 - Score: 6
40	 Chose: 2 - Score: 6
41	 Chose: 2 - Score: 1
42	 Chose: 1 - Score: 6
4

-----------------------------------

<b>c. Dự đoán viên xúc xắc được chọn</b>

Lấy chuỗi quan sát được ở câu b để dự đoán

Sử dụng thuật toán viterbi được xây dựng từ câu a.

In [81]:
observations = np.array(result)
transition_matrix = np.array(transition_matrix)
emission_matrix = np.array(emission_matrix)
pi_matrix = np.array(pi_matrix)

In [82]:
dice_pred = hmm.viterbi(V = observations, A = transition_matrix, B = emission_matrix, initial_prob = pi_matrix)

dice_pred = dice_pred.astype(np.int64)

# print out 1 or 2
print('[*] Predicted sequence: ' , [i + 1 for i in dice_pred])

[*] Predicted sequence:  [2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [84]:
wrongs = 0
for i in range(len(dice_pred)):
    wrongs += 1 if int(dice_pred[i]) != dice_chose[i] else 0

print(f'[*] Wrong: {wrongs}')
print(f'[*] Accuracy: {(len(dice_pred) - wrongs) / len(dice_pred)}') 

[*] Wrong: 28
[*] Accuracy: 0.72


Ta thực hiện lặp lại thao tác tạo một chuỗi ngẫu nhiên ở câu $b$ sau sau đó in chuỗi dự đoán ở câu $c$ rồi ghi lại vị trí của các giá trị 2 và độ chính xác của chuỗi dự đoán. Thực hiện nhiều lần, ta thu được bảng sau:

|  Lần thử  | Vị trí   | Độ chính xác |
|-------|-------|------|
|$1$|$[0]$|$0.77$|
|$2$|$[Empty]$|$0.78$|
|$3$|$[0]$|$0.77$|
|$4$|$[0]$|$0.86$|
|$5$|$[Empty]$|$0.8$|
|$6$|$[Empty]$|$0.79$|
|$7$|$[Empty]$|$0.72$|
|$8$|$[0]$|$0.76$|
|$9$|$[0]$|$0.76$|
|$10$|$[0]$|$0.73$|
|$11$|$[Empty]$|$0.76$|
|$12$|$[Empty]$|$0.75$|
|$13$|$[Empty]$|$0.79$|
|$14$|$[Empty]$|$0.72$|
|$15$|$[Empty]$|$0.78$|
|$16$|$[0, 1, 28, 29, 30, 31, 32]$|$0.8$|
|$17$|$[Empty]$|$0.77$|
|$18$|$[0]$|$0.65$|
|$19$|$[Empty]$|$0.79$|
|$20$|$[Empty]$|$0.75$|
|$Average$||$0.765$|

<b>Nhận xét</b>

Theo đề bài, rõ ràng anh Huy rất hạn chế việc sử dụng viên xúc xắc bất cân bằng (ta cũng có thể thấy từ chuỗi sinh ngẫu nhiên theo xác suất mô tả trong bài toán ở câu $b$, các giá trị 2 rất ít khi xuất hiện), vì thế việc giá trị 2 xuất hiện rất ít trong chuỗi dự đoán là điều hợp lí.

Kết quả dự đoán từ thuật toán Viterbi là khá chính xác với độ chính xác trung bình $0.765$

-------------------------------

<b>d. Giả sử bạn là một người chơi, hãy sử dụng thuật toán Baum-Welch để ước
lượng các tham số cho mô hình Markov ẩn. Hãy lặp lại thí nghiệm nhiều lần nếu cần
thiết. Báo cáo và nhận xét kết quả thu được. </b>

Với bài toán này, ta sử dụng lại chuỗi quan sát được sinh ở câu b. 

Sử dụng bảng $A$ và $B$ lần lượt là bảng chuyển đổi trạng thái ban đầu và bảng phụ thuộc trạng thái ban đầu.

In [8]:
A = [
    [0.5, 0.5],
    [0.5, 0.5]
]

B = [
    [1/6, 1/6, 1/6, 1/6, 1/6, 1/6],
    [1/6, 1/6, 1/6, 1/6, 1/6, 1/6]
]

A = np.array(A)
B = np.array(B)

A, B = hmm.baum_welch(observations, A, B, pi_matrix, 1000)

print('---------- Result ----------')
print('Matrix A:')
print(np.matrix(A.round(decimals=4)))
print('\nMatrix B:')
print(np.matrix(B.round(decimals=4)))


print('\n---------- Compare ----------')    
alpha_test = hmm.forward(observations, transition_matrix, emission_matrix, pi_matrix)
alpha_res = hmm.forward(observations, A, B, pi_matrix)

print('Likelihood with initial parameters: {}'.format(alpha_test[T - 1].sum()))
print('Likelihood with estimated parameters: {}'.format(alpha_res[T - 1].sum()))
print('Diff: {}'.format(abs(alpha_test[T - 1].sum() - alpha_res[T - 1].sum())))

---------- Result ----------
Matrix A:
[[0.5 0.5]
 [0.5 0.5]]

Matrix B:
[[0.16 0.15 0.22 0.12 0.16 0.19]
 [0.16 0.15 0.22 0.12 0.16 0.19]]

---------- Compare ----------
Likelihood with initial parameters: 6.69227948592292e-79
Likelihood with estimated parameters: 8.974301988079143e-78
Diff: 8.305074039486851e-78


<b>Nhận xét: </b> Với chuỗi quan sát với độ dài lớn như thế thì dẫn tới không gian mẫu rất lớn, do đó xác suất để có chuỗi quan sát trên là rất nhỏ, vậy nên tham số tìm được cho ra độ hợp lí (likelihood) sai lệch so với tham số ban đầu.